# Exploring the Census Datasets table

This tutorial demonstrates basic use of the `census_datasets` dataframe that contains metadata of the Census source datasets. This metadata can be joined to the cell metadata dataframe (`obs`) via the column `dataset_id`, 

**Contents**

1. Fetching the datasets table.
2. Fetching the expression data from a single dataset.
3. Downloading the original source H5AD file of a dataset.



## Fetching the datasets table


Each Census contains a top-level dataframe itemizing the datasets contained therein. You can read this into a `pandas.DataFrame`.

In [1]:
import cellxgene_census

census = cellxgene_census.open_soma()
census_datasets = census["census_info"]["datasets"].read().concat().to_pandas()

# for convenience, indexing on the soma_joinid which links this to other census data.
census_datasets = census_datasets.set_index("soma_joinid")

census_datasets

The "stable" Census version is not yet available. Using "latest" Census version instead.
The "latest" release is currently 2023-05-15. Specify 'census_version="2023-05-15"' in future calls to open_soma() to ensure data consistency.


,collection_id,collection_name,collection_doi,dataset_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
soma_joinid,,,,,,,
0,6b701826-37bb-4356-9792-ff41fc4c3161,Abdominal White Adipose Tissue,,9d8e5dca-03a3-457d-b7fb-844c75735c83,22 integrated samples,9d8e5dca-03a3-457d-b7fb-844c75735c83.h5ad,72335
1,4195ab4c-20bd-4cd3-8b3d-65601277e731,A spatially resolved single cell genomic atlas...,,a6388a6f-6076-401b-9b30-7d4306a20035,scRNA-seq data - myeloid cells,a6388a6f-6076-401b-9b30-7d4306a20035.h5ad,30789
2,4195ab4c-20bd-4cd3-8b3d-65601277e731,A spatially resolved single cell genomic atlas...,,842c6f5d-4a94-4eef-8510-8c792d1124bc,scRNA-seq data - all cells,842c6f5d-4a94-4eef-8510-8c792d1124bc.h5ad,714331
3,4195ab4c-20bd-4cd3-8b3d-65601277e731,A spatially resolved single cell genomic atlas...,,74520626-b0ba-4ee9-86b5-714649554def,scRNA-seq data - T cells,74520626-b0ba-4ee9-86b5-714649554def.h5ad,76567
4,4195ab4c-20bd-4cd3-8b3d-65601277e731,A spatially resolved single cell genomic atlas...,,396a9124-fb20-4822-bf9c-e93fdf7c999a,scRNA-seq data - B cells,396a9124-fb20-4822-bf9c-e93fdf7c999a.h5ad,12510
...,...,...,...,...,...,...,...
557,180bff9c-c8a5-4539-b13b-ddbc00d643e6,Molecular characterization of selectively vuln...,10.1038/s41593-020-00764-7,f9ad5649-f372-43e1-a3a8-423383e5a8a2,Molecular characterization of selectively vuln...,f9ad5649-f372-43e1-a3a8-423383e5a8a2.h5ad,8168
558,a72afd53-ab92-4511-88da-252fb0e26b9a,Single-cell atlas of peripheral immune respons...,10.1038/s41591-020-0944-y,456e8b9b-f872-488b-871d-94534090a865,Single-cell atlas of peripheral immune respons...,456e8b9b-f872-488b-871d-94534090a865.h5ad,44721
559,38833785-fac5-48fd-944a-0f62a4c23ed1,Construction of a human cell landscape at sing...,10.1038/s41586-020-2157-4,2adb1f8a-a6b1-4909-8ee8-484814e2d4bf,Construction of a human cell landscape at sing...,2adb1f8a-a6b1-4909-8ee8-484814e2d4bf.h5ad,598266


The sum cells across all datasets should match the number of cells across all SOMA experiments (human, mouse).

In [2]:
# Count cells across all experiments
all_experiments = (
    (organism_name, organism_experiment) for organism_name, organism_experiment in census["census_data"].items()
)
experiments_total_cells = 0
print("Count by experiment:")
for organism_name, organism_experiment in all_experiments:
    num_cells = len(organism_experiment.obs.read(column_names=["soma_joinid"]).concat().to_pandas())
    print(f"\t{num_cells} cells in {organism_name}")
    experiments_total_cells += num_cells

print(f"\nFound {experiments_total_cells} cells in all experiments.")

# Count cells across all datasets
print(f"Found {census_datasets.dataset_total_cell_count.sum()} cells in all datasets.")

Count by experiment:
	53794728 cells in homo_sapiens
	4086032 cells in mus_musculus

Found 57880760 cells in all experiments.
Found 57880760 cells in all datasets.


## Fetching the expression data from a single dataset

Lets pick one dataset to slice out of the census, and turn into an [AnnData](https://anndata.readthedocs.io/en/latest/) in-memory object. This can be used with the [ScanPy](https://scanpy.readthedocs.io/en/stable/) toolchain. You can also save this AnnData locally using the AnnData [`write`](https://anndata.readthedocs.io/en/latest/api.html#writing) API.

In [3]:
census_datasets[census_datasets.dataset_id == "0bd1a1de-3aee-40e0-b2ec-86c7a30c7149"]

,collection_id,collection_name,collection_doi,dataset_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
soma_joinid,,,,,,,
491,0b9d8a04-bb9d-44da-aa27-705bb65b54eb,Tabula Muris Senis,10.1038/s41586-020-2496-1,0bd1a1de-3aee-40e0-b2ec-86c7a30c7149,Bone marrow - A single-cell transcriptomic atl...,0bd1a1de-3aee-40e0-b2ec-86c7a30c7149.h5ad,40220


Create a query on the mouse experiment, "RNA" measurement, for the dataset_id.

In [4]:
adata = cellxgene_census.get_anndata(
    census, organism="Mus musculus", obs_value_filter="dataset_id == '0bd1a1de-3aee-40e0-b2ec-86c7a30c7149'"
)

adata

AnnData object with n_obs × n_vars = 40220 × 52392
    obs: 'soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_general', 'tissue_general_ontology_term_id'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length'

## Downloading the original source H5AD file of a dataset.

You can download the original H5AD file for any given dataset. This is the same H5AD you can download from the [CZ CELLxGENE Discover](https://cellxgene.cziscience.com/), and may contain additional data-submitter provided information which was not included in the Census.

To do this you can fetch the location in the cloud or directly download to your system using the `cellxgene-census`

In [5]:
# Option 1: Direct download
cellxgene_census.download_source_h5ad(
    "0bd1a1de-3aee-40e0-b2ec-86c7a30c7149", to_path="Tabula_Muris_Senis-bone_marrow.h5ad"
)

In [6]:
# Option 2: Get location and download via preferred method
uri = cellxgene_census.get_source_h5ad_uri("0bd1a1de-3aee-40e0-b2ec-86c7a30c7149")
uri

# you can now download the H5AD in shell via AWS CLI e.g. `aws s3 cp uri ./`

{'uri': 's3://cellxgene-data-public/cell-census/2023-05-15/h5ads/0bd1a1de-3aee-40e0-b2ec-86c7a30c7149.h5ad',
 's3_region': 'us-west-2'}

Close the census

In [7]:
census.close()